# Description

(Please, take a look at the README.md file in this directory for instructions on how to run this notebook)

This notebook compiles information about the GWAS and TWAS for a particular cohort. For example, the set of GWAS variants, variance of predicted expression of genes, etc.

It has specicfic parameters for papermill (see under `Settings` below).

This notebook is not directly run. See README.md.

# Modules

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pickle

import numpy as np

import pandas as pd
from tqdm import tqdm
import pytest

import conf
from entity import Gene

# Settings

In [3]:
# a cohort name (it could be something like UK_BIOBANK, etc)
COHORT_NAME = None

# reference panel such as 1000G or GTEX_V8
REFERENCE_PANEL = None

# predictions models such as MASHR or ELASTIC_NET
EQTL_MODEL = None

# a string with a path pointing to an imputed GWAS
GWAS_FILE = None

# a string with a path pointing where S-PrediXcan results (tissue-specific are located
SPREDIXCAN_FOLDER = None

# an f-string with one placeholder {tissue}
SPREDIXCAN_FILE_PATTERN = None

# a string with a path pointing to an S-MultiXcan result
SMULTIXCAN_FILE = None

In [4]:
# Parameters
COHORT_NAME = "1000G_EUR"
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"
GWAS_FILE = "/opt/data/results/gls/null_sims/final_imputed_gwas/random.pheno0.glm-imputed.txt.gz"
SPREDIXCAN_FOLDER = "/opt/data/results/gls/null_sims/twas/spredixcan/"
SPREDIXCAN_FILE_PATTERN = "random.pheno0-gtex_v8-mashr-{tissue}.csv"
SMULTIXCAN_FILE = "/opt/data/results/gls/null_sims/twas/smultixcan/random.pheno0-gtex_v8-mashr-smultixcan.txt"


In [5]:
assert COHORT_NAME is not None and len(COHORT_NAME) > 0, "A cohort name must be given"

COHORT_NAME = COHORT_NAME.lower()
display(f"Cohort name: {COHORT_NAME}")

'Cohort name: 1000g_eur'

In [6]:
assert (
    REFERENCE_PANEL is not None and len(REFERENCE_PANEL) > 0
), "A reference panel must be given"

display(f"Reference panel: {REFERENCE_PANEL}")

'Reference panel: GTEX_V8'

In [7]:
assert GWAS_FILE is not None and len(GWAS_FILE) > 0, "A GWAS file path must be given"
GWAS_FILE = Path(GWAS_FILE).resolve()
assert GWAS_FILE.exists(), "GWAS file does not exist"

display(f"GWAS file path: {str(GWAS_FILE)}")

'GWAS file path: /opt/data/results/gls/null_sims/final_imputed_gwas/random.pheno0.glm-imputed.txt.gz'

In [8]:
assert (
    SPREDIXCAN_FOLDER is not None and len(SPREDIXCAN_FOLDER) > 0
), "An S-PrediXcan folder path must be given"
SPREDIXCAN_FOLDER = Path(SPREDIXCAN_FOLDER).resolve()
assert SPREDIXCAN_FOLDER.exists(), "S-PrediXcan folder does not exist"

display(f"S-PrediXcan folder path: {str(SPREDIXCAN_FOLDER)}")

'S-PrediXcan folder path: /opt/data/results/gls/null_sims/twas/spredixcan'

In [9]:
assert (
    SPREDIXCAN_FILE_PATTERN is not None and len(SPREDIXCAN_FILE_PATTERN) > 0
), "An S-PrediXcan file pattern must be given"
assert (
    "{tissue}" in SPREDIXCAN_FILE_PATTERN
), "S-PrediXcan file pattern must have a '{tissue}' placeholder"

display(f"S-PrediXcan file template: {SPREDIXCAN_FILE_PATTERN}")

'S-PrediXcan file template: random.pheno0-gtex_v8-mashr-{tissue}.csv'

In [10]:
assert (
    SMULTIXCAN_FILE is not None and len(SMULTIXCAN_FILE) > 0
), "An S-MultiXcan result file path must be given"
SMULTIXCAN_FILE = Path(SMULTIXCAN_FILE).resolve()
assert SMULTIXCAN_FILE.exists(), "S-MultiXcan result file does not exist"

display(f"S-MultiXcan file path: {str(SMULTIXCAN_FILE)}")

'S-MultiXcan file path: /opt/data/results/gls/null_sims/twas/smultixcan/random.pheno0-gtex_v8-mashr-smultixcan.txt'

In [11]:
assert (
    EQTL_MODEL is not None and len(EQTL_MODEL) > 0
), "A prediction/eQTL model must be given"

display(f"eQTL model: {EQTL_MODEL}")

'eQTL model: MASHR'

In [12]:
OUTPUT_DIR_BASE = (
    conf.RESULTS["GLS"]
    / "gene_corrs"
    / "cohorts"
    / COHORT_NAME
    / REFERENCE_PANEL.lower()
    / EQTL_MODEL.lower()
)

OUTPUT_DIR_BASE.mkdir(parents=True, exist_ok=True)

display(f"Using output dir base: {OUTPUT_DIR_BASE}")

'Using output dir base: /opt/data/results/gls/gene_corrs/cohorts/1000g_eur/gtex_v8/mashr'

# Load MultiPLIER Z genes

In [13]:
multiplier_z_genes = pd.read_pickle(
    conf.MULTIPLIER["MODEL_Z_MATRIX_FILE"]
).index.tolist()

In [14]:
len(multiplier_z_genes)

6750

In [15]:
assert len(multiplier_z_genes) == len(set(multiplier_z_genes))

In [16]:
multiplier_z_genes[:5]

['GAS6', 'MMP14', 'DSP', 'MARCKSL1', 'SPARC']

# GWAS

In [17]:
gwas_file_columns = pd.read_csv(GWAS_FILE, sep="\t", nrows=2).columns
assert (
    "panel_variant_id" in gwas_file_columns
), "GWAS file must be final imputed one with column 'panel_variant_id'"
# FIXME: add other needed columns here

In [18]:
gwas_data = pd.read_csv(
    GWAS_FILE,
    sep="\t",
    usecols=["panel_variant_id", "pvalue", "zscore", "imputation_status"],
)

In [19]:
gwas_data.shape

(8325729, 4)

In [20]:
gwas_data.head()

,panel_variant_id,zscore,pvalue,imputation_status
0,chr1_54490_G_A_b38,1.567452,0.117009,original
1,chr1_87021_T_C_b38,0.244372,0.806943,imputed
2,chr1_263722_C_G_b38,1.155239,0.247993,imputed
3,chr1_594402_C_T_b38,0.851234,0.394639,imputed
4,chr1_630555_C_T_b38,0.855129,0.392480,imputed


In [21]:
gwas_data["imputation_status"].unique()

array(['original', 'imputed'], dtype=object)

In [22]:
gwas_data.dropna().shape

(8325729, 4)

In [23]:
# remove SNPs with no results
gwas_data = gwas_data.dropna()

In [24]:
gwas_data.shape

(8325729, 4)

## Save GWAS variants

In [25]:
gwas_data.head()

,panel_variant_id,zscore,pvalue,imputation_status
0,chr1_54490_G_A_b38,1.567452,0.117009,original
1,chr1_87021_T_C_b38,0.244372,0.806943,imputed
2,chr1_263722_C_G_b38,1.155239,0.247993,imputed
3,chr1_594402_C_T_b38,0.851234,0.394639,imputed
4,chr1_630555_C_T_b38,0.855129,0.392480,imputed


In [26]:
assert gwas_data["panel_variant_id"].is_unique

In [27]:
gwas_variants_ids_set = frozenset(gwas_data["panel_variant_id"])
list(gwas_variants_ids_set)[:5]

['chr2_115338583_C_G_b38',
 'chr5_178822592_T_C_b38',
 'chr7_26587172_T_C_b38',
 'chr1_55345813_G_C_b38',
 'chr7_118912308_G_T_b38']

In [28]:
with open(OUTPUT_DIR_BASE / "gwas_variant_ids.pkl", "wb") as handle:
    pickle.dump(gwas_variants_ids_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

# TWAS

## Available tissues for eQTL model

In [29]:
prediction_model_tissues = conf.PHENOMEXCAN["PREDICTION_MODELS"][
    f"{EQTL_MODEL}_TISSUES"
].split(" ")

In [30]:
len(prediction_model_tissues)

49

In [31]:
prediction_model_tissues[:5]

['Skin_Not_Sun_Exposed_Suprapubic',
 'Cells_EBV-transformed_lymphocytes',
 'Brain_Frontal_Cortex_BA9',
 'Kidney_Cortex',
 'Brain_Substantia_nigra']

## S-MultiXcan results

In [32]:
smultixcan_results = pd.read_csv(
    SMULTIXCAN_FILE, sep="\t", usecols=["gene", "gene_name", "pvalue", "n", "n_indep"]
)

In [33]:
smultixcan_results.shape

(22317, 5)

In [34]:
smultixcan_results = smultixcan_results.dropna()

In [35]:
smultixcan_results.shape

(22314, 5)

In [36]:
smultixcan_results = smultixcan_results.assign(
    gene_id=smultixcan_results["gene"].apply(lambda g: g.split(".")[0])
)

In [37]:
smultixcan_results.head()

,gene,gene_name,pvalue,n,n_indep,gene_id
0,ENSG00000131941.7,RHPN2,0.000040,48.0,3.0,ENSG00000131941
1,ENSG00000076650.6,GPATCH1,0.000078,40.0,3.0,ENSG00000076650
2,ENSG00000100906.10,NFKBIA,0.000096,1.0,1.0,ENSG00000100906
3,ENSG00000136319.11,TTC5,0.000109,47.0,5.0,ENSG00000136319
4,ENSG00000152990.13,ADGRA3,0.000135,41.0,12.0,ENSG00000152990


In [38]:
assert smultixcan_results["gene_id"].is_unique

### Remove duplicated gene names

In [39]:
smultixcan_results["gene_name"].is_unique

False

In [40]:
# list duplicated gene names
_smultixcan_duplicated_gene_names = smultixcan_results[
    smultixcan_results["gene_name"].duplicated(keep=False)
]
display(_smultixcan_duplicated_gene_names)

,gene,gene_name,pvalue,n,n_indep,gene_id
1463,ENSG00000235641.4,LINC00484,0.068450,35.0,3.0,ENSG00000235641
2393,ENSG00000237667.5,LINC01115,0.112329,22.0,1.0,ENSG00000237667
5200,ENSG00000272342.1,LINC01115,0.243122,4.0,1.0,ENSG00000272342
5884,ENSG00000229694.6,LINC00484,0.274814,40.0,5.0,ENSG00000229694
6131,ENSG00000147676.13,MAL2,0.286003,29.0,5.0,ENSG00000147676
6209,ENSG00000283992.1,LYNX1,0.289628,45.0,3.0,ENSG00000283992
10560,ENSG00000180155.19,LYNX1,0.486960,48.0,3.0,ENSG00000180155
14028,ENSG00000235271.5,LINC01422,0.643355,38.0,6.0,ENSG00000235271
14253,ENSG00000253972.5,MAL2,0.654873,6.0,4.0,ENSG00000253972
20085,ENSG00000182957.15,SPATA13,0.906729,40.0,14.0,ENSG00000182957


In [41]:
# TODO: my strategy below to handle duplicated gene names is to keep the first one
#  it might be better to have another strategy, maybe keeping the most significant

In [42]:
smultixcan_results = smultixcan_results.drop_duplicates(
    subset=["gene_name"], keep="first"
)
display(smultixcan_results.shape)

(22308, 6)

### Get common genes with MultiPLIER

In [43]:
common_genes = set(multiplier_z_genes).intersection(
    set(smultixcan_results["gene_name"])
)

In [44]:
len(common_genes)

6444

In [45]:
sorted(list(common_genes))[:5]

['A2M', 'AAAS', 'AANAT', 'AARS', 'AARS2']

In [46]:
assert smultixcan_results[smultixcan_results["gene_name"].isin(common_genes)].shape[
    0
] == len(common_genes)

In [47]:
smultixcan_gene_id_common = smultixcan_results[
    smultixcan_results["gene_name"].isin(common_genes)
]["gene_id"]

In [48]:
smultixcan_gene_id_common.shape

(6444,)

In [49]:
assert smultixcan_gene_id_common.is_unique

In [50]:
smultixcan_gene_id_common = set(smultixcan_gene_id_common)

### Save

In [51]:
with open(OUTPUT_DIR_BASE / "common_genes.pkl", "wb") as handle:
    pickle.dump(common_genes, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Genes info

In [52]:
multiplier_gene_obj = {
    gene_name: Gene(name=gene_name)
    for gene_name in common_genes
    if gene_name in Gene.GENE_NAME_TO_ID_MAP
}

In [53]:
len(multiplier_gene_obj)

6444

In [54]:
assert multiplier_gene_obj["GAS6"].ensembl_id == "ENSG00000183087"

In [55]:
_gene_obj = list(multiplier_gene_obj.values())

genes_info = pd.DataFrame(
    {
        "name": [g.name for g in _gene_obj],
        "id": [g.ensembl_id for g in _gene_obj],
        "chr": [g.chromosome for g in _gene_obj],
        "band": [g.band for g in _gene_obj],
        "start_position": [g.get_attribute("start_position") for g in _gene_obj],
        "end_position": [g.get_attribute("end_position") for g in _gene_obj],
    }
)

In [56]:
genes_info = genes_info.assign(
    gene_length=genes_info.apply(
        lambda x: x["end_position"] - x["start_position"], axis=1
    )
)

In [57]:
genes_info.shape

(6444, 7)

In [58]:
genes_info.head()

,name,id,chr,band,start_position,end_position,gene_length
0,MAPK11,ENSG00000185386,22,22q13.33,50263713.0,50270767.0,7054.0
1,RUNX1,ENSG00000159216,21,21q22.12,34787801.0,36004667.0,1216866.0
2,RFC4,ENSG00000163918,3,3q27.3,186789880.0,186807058.0,17178.0
3,ANG,ENSG00000214274,14,14q11.2,20684177.0,20698971.0,14794.0
4,NKD1,ENSG00000140807,16,16q12.1,50548396.0,50649249.0,100853.0


In [59]:
genes_info.sort_values("chr")

,name,id,chr,band,start_position,end_position,gene_length
2664,CYP2J2,ENSG00000134716,1,1p32.1,59893308.0,59926773.0,33465.0
3965,SH3GLB1,ENSG00000097033,1,1p22.3,86704570.0,86748184.0,43614.0
4432,AMPD2,ENSG00000116337,1,1p13.3,109616104.0,109632051.0,15947.0
1050,CAMTA1,ENSG00000171735,1,1p36.31,6785454.0,7769706.0,984252.0
5327,TAL1,ENSG00000162367,1,1p33,47216290.0,47232220.0,15930.0
...,...,...,...,...,...,...,...
505,CLTA,ENSG00000122705,9,9p13.3,36190856.0,36304781.0,113925.0
2812,ARPC5L,ENSG00000136950,9,9q33.3,124862130.0,124877733.0,15603.0
3813,PSMB7,ENSG00000136930,9,9q33.3,124353465.0,124415444.0,61979.0
336,TMEM133,ENSG00000170647,None,None,NaN,NaN,NaN


### Save

In [60]:
genes_info.to_pickle(OUTPUT_DIR_BASE / "genes_info.pkl")

## S-PrediXcan results

### Load results across all tissues

In [61]:
spredixcan_result_files = {
    t: SPREDIXCAN_FOLDER / SPREDIXCAN_FILE_PATTERN.format(tissue=t)
    for t in prediction_model_tissues
}

In [62]:
assert len(spredixcan_result_files) == len(prediction_model_tissues)
display(list(spredixcan_result_files.values())[:5])

[PosixPath('/opt/data/results/gls/null_sims/twas/spredixcan/random.pheno0-gtex_v8-mashr-Skin_Not_Sun_Exposed_Suprapubic.csv'),
 PosixPath('/opt/data/results/gls/null_sims/twas/spredixcan/random.pheno0-gtex_v8-mashr-Cells_EBV-transformed_lymphocytes.csv'),
 PosixPath('/opt/data/results/gls/null_sims/twas/spredixcan/random.pheno0-gtex_v8-mashr-Brain_Frontal_Cortex_BA9.csv'),
 PosixPath('/opt/data/results/gls/null_sims/twas/spredixcan/random.pheno0-gtex_v8-mashr-Kidney_Cortex.csv'),
 PosixPath('/opt/data/results/gls/null_sims/twas/spredixcan/random.pheno0-gtex_v8-mashr-Brain_Substantia_nigra.csv')]

In [63]:
# look at the structure of one result
pd.read_csv(spredixcan_result_files["Whole_Blood"]).head()

,gene,gene_name,zscore,effect_size,pvalue,var_g,pred_perf_r2,pred_perf_pval,pred_perf_qval,n_snps_used,n_snps_in_cov,n_snps_in_model,best_gwas_p,largest_weight
0,ENSG00000131236.16,CAP1,-3.882684,NaN,0.000103,8.369900e-05,NaN,NaN,NaN,1,1,1,0.000103,0.021593
1,ENSG00000130787.13,HIP1R,3.862583,NaN,0.000112,3.039669e-03,NaN,NaN,NaN,2,2,2,0.001681,0.087495
2,ENSG00000103018.16,CYB5B,3.478682,NaN,0.000504,6.455623e-03,NaN,NaN,NaN,1,1,1,0.000504,0.114291
3,ENSG00000117906.13,RCN2,3.431258,NaN,0.000601,5.848182e-07,NaN,NaN,NaN,1,1,1,0.000601,0.001696
4,ENSG00000135469.13,COQ10A,3.363631,NaN,0.000769,4.312235e-04,NaN,NaN,NaN,2,2,2,0.013565,0.064460


In [64]:
assert all(f.exists() for f in spredixcan_result_files.values())

In [65]:
spredixcan_dfs = [
    pd.read_csv(
        f,
        usecols=[
            "gene",
            "gene_name",
            "zscore",
            "pvalue",
            "n_snps_used",
            "n_snps_in_model",
        ],
    )
    .dropna(subset=["gene", "zscore", "pvalue"])
    .assign(tissue=t)
    for t, f in spredixcan_result_files.items()
]

In [66]:
assert len(spredixcan_dfs) == len(prediction_model_tissues)

In [67]:
spredixcan_dfs = pd.concat(spredixcan_dfs)

In [68]:
assert spredixcan_dfs["tissue"].unique().shape[0] == len(prediction_model_tissues)

In [69]:
spredixcan_dfs.shape

(652858, 7)

In [70]:
spredixcan_dfs = spredixcan_dfs.assign(
    gene_id=spredixcan_dfs["gene"].apply(lambda g: g.split(".")[0])
)

In [71]:
spredixcan_dfs.head()

,gene,gene_name,zscore,pvalue,n_snps_used,n_snps_in_model,tissue,gene_id
0,ENSG00000131236.16,CAP1,-3.882684,0.000103,1,1,Skin_Not_Sun_Exposed_Suprapubic,ENSG00000131236
1,ENSG00000196172.9,ZNF681,-3.710444,0.000207,1,1,Skin_Not_Sun_Exposed_Suprapubic,ENSG00000196172
2,ENSG00000280789.1,PAGR1,3.403263,0.000666,2,2,Skin_Not_Sun_Exposed_Suprapubic,ENSG00000280789
3,ENSG00000215218.3,UBE2QL1,3.385093,0.000712,3,3,Skin_Not_Sun_Exposed_Suprapubic,ENSG00000215218
4,ENSG00000167371.17,PRRT2,3.381834,0.000720,2,2,Skin_Not_Sun_Exposed_Suprapubic,ENSG00000167371


In [72]:
# leave only common genes
spredixcan_dfs = spredixcan_dfs[
    spredixcan_dfs["gene_id"].isin(smultixcan_gene_id_common)
].drop(columns=["gene_name"])

In [73]:
spredixcan_dfs.shape

(233445, 7)

### Count number of tissues available per gene

In [74]:
spredixcan_genes_n_models = spredixcan_dfs.groupby("gene_id")["tissue"].nunique()

In [75]:
spredixcan_genes_n_models

gene_id
ENSG00000000419     2
ENSG00000000938    36
ENSG00000000971    34
ENSG00000001084    32
ENSG00000001167    40
                   ..
ENSG00000278540    36
ENSG00000278828     4
ENSG00000278845    49
ENSG00000281005    49
ENSG00000282608    36
Name: tissue, Length: 6444, dtype: int64

In [76]:
# testing that in S-MultiXcan I get the same number of tissues per gene
_tmp_smultixcan_results_n_models = (
    smultixcan_results.set_index("gene_id")["n"].astype(int).rename("tissue")
)

_cg = _tmp_smultixcan_results_n_models.index.intersection(
    spredixcan_genes_n_models.index
)
_tmp_smultixcan_results_n_models = _tmp_smultixcan_results_n_models.loc[_cg]
_spredixcan = spredixcan_genes_n_models.loc[_cg]

assert _spredixcan.shape[0] == _tmp_smultixcan_results_n_models.shape[0]
assert _spredixcan.equals(_tmp_smultixcan_results_n_models.loc[_spredixcan.index])

### Get tissues available per gene

In [77]:
spredixcan_genes_models = spredixcan_dfs.groupby("gene_id")["tissue"].apply(
    lambda x: frozenset(x.tolist())
)

In [78]:
spredixcan_genes_models

gene_id
ENSG00000000419         (Brain_Hypothalamus, Brain_Substantia_nigra)
ENSG00000000938    (Breast_Mammary_Tissue, Esophagus_Mucosa, Pros...
ENSG00000000971    (Breast_Mammary_Tissue, Stomach, Artery_Corona...
ENSG00000001084    (Pancreas, Artery_Coronary, Lung, Brain_Putame...
ENSG00000001167    (Breast_Mammary_Tissue, Esophagus_Mucosa, Pros...
                                         ...                        
ENSG00000278540    (Breast_Mammary_Tissue, Esophagus_Mucosa, Pros...
ENSG00000278828    (Cells_EBV-transformed_lymphocytes, Adipose_Vi...
ENSG00000278845    (Breast_Mammary_Tissue, Esophagus_Mucosa, Pros...
ENSG00000281005    (Breast_Mammary_Tissue, Esophagus_Mucosa, Pros...
ENSG00000282608    (Breast_Mammary_Tissue, Esophagus_Mucosa, Pros...
Name: tissue, Length: 6444, dtype: object

In [79]:
assert spredixcan_genes_n_models.shape[0] == spredixcan_genes_models.shape[0]

In [80]:
assert spredixcan_genes_n_models.index.equals(spredixcan_genes_models.index)

In [81]:
assert (spredixcan_genes_models.apply(len) <= len(prediction_model_tissues)).all()

In [82]:
spredixcan_genes_models.apply(len).describe()

count    6444.000000
mean       36.226723
std        12.973688
min         1.000000
25%        29.000000
50%        41.000000
75%        47.000000
max        49.000000
Name: tissue, dtype: float64

In [83]:
# testing
assert (
    spredixcan_genes_models.loc[spredixcan_genes_n_models.index]
    .apply(len)
    .equals(spredixcan_genes_n_models)
)

### Add gene name and set index

In [84]:
spredixcan_genes_models = spredixcan_genes_models.to_frame().reset_index()

In [85]:
spredixcan_genes_models.head()

,gene_id,tissue
0,ENSG00000000419,"(Brain_Hypothalamus, Brain_Substantia_nigra)"
1,ENSG00000000938,"(Breast_Mammary_Tissue, Esophagus_Mucosa, Pros..."
2,ENSG00000000971,"(Breast_Mammary_Tissue, Stomach, Artery_Corona..."
3,ENSG00000001084,"(Pancreas, Artery_Coronary, Lung, Brain_Putame..."
4,ENSG00000001167,"(Breast_Mammary_Tissue, Esophagus_Mucosa, Pros..."


In [86]:
spredixcan_genes_models = spredixcan_genes_models.assign(
    gene_name=spredixcan_genes_models["gene_id"].apply(
        lambda g: Gene.GENE_ID_TO_NAME_MAP[g]
    )
)

In [87]:
spredixcan_genes_models = spredixcan_genes_models[["gene_id", "gene_name", "tissue"]]

In [88]:
spredixcan_genes_models = spredixcan_genes_models.set_index("gene_id")

In [89]:
spredixcan_genes_models.head()

,gene_name,tissue
gene_id,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)"
ENSG00000000938,FGR,"(Breast_Mammary_Tissue, Esophagus_Mucosa, Pros..."
ENSG00000000971,CFH,"(Breast_Mammary_Tissue, Stomach, Artery_Corona..."
ENSG00000001084,GCLC,"(Pancreas, Artery_Coronary, Lung, Brain_Putame..."
ENSG00000001167,NFYA,"(Breast_Mammary_Tissue, Esophagus_Mucosa, Pros..."


### Add number of tissues

In [90]:
spredixcan_genes_models = spredixcan_genes_models.assign(
    n_tissues=spredixcan_genes_models["tissue"].apply(len)
)

In [91]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues
gene_id,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2
ENSG00000000938,FGR,"(Breast_Mammary_Tissue, Esophagus_Mucosa, Pros...",36
ENSG00000000971,CFH,"(Breast_Mammary_Tissue, Stomach, Artery_Corona...",34
ENSG00000001084,GCLC,"(Pancreas, Artery_Coronary, Lung, Brain_Putame...",32
ENSG00000001167,NFYA,"(Breast_Mammary_Tissue, Esophagus_Mucosa, Pros...",40


### Get gene's objects

In [92]:
spredixcan_gene_obj = {
    gene_id: Gene(ensembl_id=gene_id) for gene_id in spredixcan_genes_models.index
}

In [93]:
len(spredixcan_gene_obj)

6444

### Add genes' variance captured by principal components

In [94]:
def _get_gene_pc_variance(gene_row):
    gene_id = gene_row.name
    gene_tissues = gene_row["tissue"]
    gene_obj = spredixcan_gene_obj[gene_id]

    u, s, vt = gene_obj.get_tissues_correlations_svd(
        tissues=gene_tissues,
        snps_subset=gwas_variants_ids_set,
        reference_panel=REFERENCE_PANEL,
        model_type=EQTL_MODEL,
        # use_covariance_matrix=True,
    )

    return s

In [95]:
_tmp = spredixcan_genes_models.loc["ENSG00000188976"]
_get_gene_pc_variance(_tmp)

array([35.14301823,  3.87568728,  2.3091285 ,  1.90683623,  1.27573165])

In [96]:
spredixcan_genes_tissues_pc_variance = spredixcan_genes_models.apply(
    _get_gene_pc_variance, axis=1
)

In [97]:
spredixcan_genes_tissues_pc_variance

gene_id
ENSG00000000419             [1.0442398458252409, 0.9557601541747592]
ENSG00000000938    [31.63266933637128, 2.0783677518511436, 1.2716...
ENSG00000000971    [21.560769925683296, 7.3101709870454625, 1.837...
ENSG00000001084    [21.291939080546523, 4.699418748743197, 2.2489...
ENSG00000001167              [37.33566029549769, 1.3914019604538364]
                                         ...                        
ENSG00000278540    [30.239540272954745, 3.0448326465798896, 1.679...
ENSG00000278828              [3.062182335109643, 0.9378176648903567]
ENSG00000278845             [45.687454510139105, 2.2268019546482347]
ENSG00000281005                                 [48.301707397855736]
ENSG00000282608    [22.653514081237095, 6.245520030604542, 2.5559...
Length: 6444, dtype: object

In [98]:
# # testing
# assert spredixcan_genes_tissues_pc_variance.loc[
#     "ENSG00000188976"
# ].sum() == pytest.approx(44.01605629086847)
# # this is using the covariance:
# # assert spredixcan_genes_tissues_pc_variance.loc["ENSG00000188976"].sum() == pytest.approx(1.1492946006449425)

In [99]:
# add to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_tissues_pc_variance.rename("tissues_pc_variances")
)

In [100]:
spredixcan_genes_models.shape

(6444, 4)

In [101]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances
gene_id,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,"[1.0442398458252409, 0.9557601541747592]"
ENSG00000000938,FGR,"(Breast_Mammary_Tissue, Esophagus_Mucosa, Pros...",36,"[31.63266933637128, 2.0783677518511436, 1.2716..."
ENSG00000000971,CFH,"(Breast_Mammary_Tissue, Stomach, Artery_Corona...",34,"[21.560769925683296, 7.3101709870454625, 1.837..."
ENSG00000001084,GCLC,"(Pancreas, Artery_Coronary, Lung, Brain_Putame...",32,"[21.291939080546523, 4.699418748743197, 2.2489..."
ENSG00000001167,NFYA,"(Breast_Mammary_Tissue, Esophagus_Mucosa, Pros...",40,"[37.33566029549769, 1.3914019604538364]"


### Add genes' variance captured by principal components (covariance)

In [102]:
def _get_gene_pc_variance(gene_row):
    gene_id = gene_row.name
    gene_tissues = gene_row["tissue"]
    gene_obj = spredixcan_gene_obj[gene_id]

    u, s, vt = gene_obj.get_tissues_correlations_svd(
        tissues=gene_tissues,
        snps_subset=gwas_variants_ids_set,
        reference_panel=REFERENCE_PANEL,
        model_type=EQTL_MODEL,
        use_covariance_matrix=True,
    )

    return s

In [103]:
_tmp = spredixcan_genes_models.loc["ENSG00000188976"]
_get_gene_pc_variance(_tmp)

array([0.92569322, 0.08878404])

In [104]:
spredixcan_genes_tissues_pc_variance = spredixcan_genes_models.apply(
    _get_gene_pc_variance, axis=1
)

In [105]:
spredixcan_genes_tissues_pc_variance

gene_id
ENSG00000000419       [0.013163142634155849, 0.00048574890347684763]
ENSG00000000938                                [0.17159486570780277]
ENSG00000000971    [0.2093060319587451, 0.058234347656008094, 0.0...
ENSG00000001084    [0.39430527375123775, 0.11627488888534823, 0.0...
ENSG00000001167            [1.1642639979821048, 0.05742881287671711]
                                         ...                        
ENSG00000278540           [0.1210517732183371, 0.011043537807199556]
ENSG00000278828        [0.003774008788117112, 0.0024289076167174424]
ENSG00000278845            [1.0913874047417143, 0.04698044450101885]
ENSG00000281005                                 [3.8662049265278142]
ENSG00000282608    [0.22315950079956806, 0.031779000782788114, 0....
Length: 6444, dtype: object

In [106]:
# # testing
# # assert spredixcan_genes_tissues_pc_variance.loc["ENSG00000188976"].sum() == pytest.approx(44.01605629086847)
# # this is using the covariance:
# assert spredixcan_genes_tissues_pc_variance.loc[
#     "ENSG00000188976"
# ].sum() == pytest.approx(1.1492946006449425)

In [107]:
# add to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_tissues_pc_variance.rename("tissues_pc_variances_cov")
)

In [108]:
spredixcan_genes_models.shape

(6444, 5)

In [109]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_pc_variances_cov
gene_id,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,"[1.0442398458252409, 0.9557601541747592]","[0.013163142634155849, 0.00048574890347684763]"
ENSG00000000938,FGR,"(Breast_Mammary_Tissue, Esophagus_Mucosa, Pros...",36,"[31.63266933637128, 2.0783677518511436, 1.2716...",[0.17159486570780277]
ENSG00000000971,CFH,"(Breast_Mammary_Tissue, Stomach, Artery_Corona...",34,"[21.560769925683296, 7.3101709870454625, 1.837...","[0.2093060319587451, 0.058234347656008094, 0.0..."
ENSG00000001084,GCLC,"(Pancreas, Artery_Coronary, Lung, Brain_Putame...",32,"[21.291939080546523, 4.699418748743197, 2.2489...","[0.39430527375123775, 0.11627488888534823, 0.0..."
ENSG00000001167,NFYA,"(Breast_Mammary_Tissue, Esophagus_Mucosa, Pros...",40,"[37.33566029549769, 1.3914019604538364]","[1.1642639979821048, 0.05742881287671711]"


### Add gene variance per tissue

In [110]:
def _get_gene_variances(gene_row):
    gene_id = gene_row.name
    gene_tissues = gene_row["tissue"]

    tissue_variances = {}
    gene_obj = spredixcan_gene_obj[gene_id]

    for tissue in gene_tissues:
        tissue_var = gene_obj.get_pred_expression_variance(
            tissue=tissue,
            reference_panel=REFERENCE_PANEL,
            model_type=EQTL_MODEL,
            snps_subset=gwas_variants_ids_set,
        )

        if tissue_var is not None:
            tissue_variances[tissue] = tissue_var

    return tissue_variances

In [111]:
_tmp = spredixcan_genes_models.loc["ENSG00000000419"]
_get_gene_variances(_tmp)

{'Brain_Hypothalamus': 0.013162153504206677,
 'Brain_Substantia_nigra': 0.0004867380334260178}

In [112]:
spredixcan_genes_tissues_variance = spredixcan_genes_models.apply(
    _get_gene_variances, axis=1
)

In [113]:
spredixcan_genes_tissues_variance

gene_id
ENSG00000000419    {'Brain_Hypothalamus': 0.013162153504206677, '...
ENSG00000000938    {'Breast_Mammary_Tissue': 0.002243819216617953...
ENSG00000000971    {'Breast_Mammary_Tissue': 0.01068561006340443,...
ENSG00000001084    {'Pancreas': 0.015199520517598328, 'Artery_Cor...
ENSG00000001167    {'Breast_Mammary_Tissue': 0.03313519857456931,...
                                         ...                        
ENSG00000278540    {'Breast_Mammary_Tissue': 0.002812789859872168...
ENSG00000278828    {'Cells_EBV-transformed_lymphocytes': 0.003027...
ENSG00000278845    {'Breast_Mammary_Tissue': 0.007262227905134688...
ENSG00000281005    {'Breast_Mammary_Tissue': 0.03229355035207454,...
ENSG00000282608    {'Breast_Mammary_Tissue': 0.005205092662061717...
Length: 6444, dtype: object

In [114]:
# # testing
# _gene_id = "ENSG00000188976"
# x = spredixcan_genes_tissues_variance.loc[_gene_id]
# # expected value obtained by sum of PCA eigenvalues on this gene's predicted expression
# assert np.sum(list(x.values())) == pytest.approx(1.2326202607409493)

In [115]:
# testing
spredixcan_genes_tissues_variance.loc["ENSG00000000419"]

{'Brain_Hypothalamus': 0.013162153504206677,
 'Brain_Substantia_nigra': 0.0004867380334260178}

In [116]:
# FIXME: maybe add more tests, these differt from GTEX V8
# # testing
# # here values were obtained from S-PrediXcan results, where the reference panel is GTEX V8, not 1000G, so just approximations)
# _gene_id = "ENSG00000000419"
# assert spredixcan_genes_tissues_variance.loc[_gene_id]["Brain_Substantia_nigra"] == pytest.approx(0.0004266255268163448)
# assert spredixcan_genes_tissues_variance.loc[_gene_id]["Brain_Hypothalamus"] == pytest.approx(0.011235877515236132)

In [117]:
# add to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_tissues_variance.rename("tissues_variances")
)

In [118]:
spredixcan_genes_models.shape

(6444, 6)

In [119]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_pc_variances_cov,tissues_variances
gene_id,,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,"[1.0442398458252409, 0.9557601541747592]","[0.013163142634155849, 0.00048574890347684763]","{'Brain_Hypothalamus': 0.013162153504206677, '..."
ENSG00000000938,FGR,"(Breast_Mammary_Tissue, Esophagus_Mucosa, Pros...",36,"[31.63266933637128, 2.0783677518511436, 1.2716...",[0.17159486570780277],{'Breast_Mammary_Tissue': 0.002243819216617953...
ENSG00000000971,CFH,"(Breast_Mammary_Tissue, Stomach, Artery_Corona...",34,"[21.560769925683296, 7.3101709870454625, 1.837...","[0.2093060319587451, 0.058234347656008094, 0.0...","{'Breast_Mammary_Tissue': 0.01068561006340443,..."
ENSG00000001084,GCLC,"(Pancreas, Artery_Coronary, Lung, Brain_Putame...",32,"[21.291939080546523, 4.699418748743197, 2.2489...","[0.39430527375123775, 0.11627488888534823, 0.0...","{'Pancreas': 0.015199520517598328, 'Artery_Cor..."
ENSG00000001167,NFYA,"(Breast_Mammary_Tissue, Esophagus_Mucosa, Pros...",40,"[37.33566029549769, 1.3914019604538364]","[1.1642639979821048, 0.05742881287671711]","{'Breast_Mammary_Tissue': 0.03313519857456931,..."


### Count number of SNPs predictors used across tissue models

In [120]:
spredixcan_genes_sum_of_n_snps_used = (
    spredixcan_dfs.groupby("gene_id")["n_snps_used"].sum().rename("n_snps_used_sum")
)

In [121]:
spredixcan_genes_sum_of_n_snps_used

gene_id
ENSG00000000419     2
ENSG00000000938    40
ENSG00000000971    44
ENSG00000001084    46
ENSG00000001167    47
                   ..
ENSG00000278540    44
ENSG00000278828     5
ENSG00000278845    89
ENSG00000281005    81
ENSG00000282608    40
Name: n_snps_used_sum, Length: 6444, dtype: int64

In [122]:
# add sum of snps used to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_sum_of_n_snps_used
)

In [123]:
spredixcan_genes_models.shape

(6444, 7)

In [124]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_pc_variances_cov,tissues_variances,n_snps_used_sum
gene_id,,,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,"[1.0442398458252409, 0.9557601541747592]","[0.013163142634155849, 0.00048574890347684763]","{'Brain_Hypothalamus': 0.013162153504206677, '...",2
ENSG00000000938,FGR,"(Breast_Mammary_Tissue, Esophagus_Mucosa, Pros...",36,"[31.63266933637128, 2.0783677518511436, 1.2716...",[0.17159486570780277],{'Breast_Mammary_Tissue': 0.002243819216617953...,40
ENSG00000000971,CFH,"(Breast_Mammary_Tissue, Stomach, Artery_Corona...",34,"[21.560769925683296, 7.3101709870454625, 1.837...","[0.2093060319587451, 0.058234347656008094, 0.0...","{'Breast_Mammary_Tissue': 0.01068561006340443,...",44
ENSG00000001084,GCLC,"(Pancreas, Artery_Coronary, Lung, Brain_Putame...",32,"[21.291939080546523, 4.699418748743197, 2.2489...","[0.39430527375123775, 0.11627488888534823, 0.0...","{'Pancreas': 0.015199520517598328, 'Artery_Cor...",46
ENSG00000001167,NFYA,"(Breast_Mammary_Tissue, Esophagus_Mucosa, Pros...",40,"[37.33566029549769, 1.3914019604538364]","[1.1642639979821048, 0.05742881287671711]","{'Breast_Mammary_Tissue': 0.03313519857456931,...",47


### Count number of SNPs predictors in models across tissue models

In [125]:
spredixcan_genes_sum_of_n_snps_in_model = (
    spredixcan_dfs.groupby("gene_id")["n_snps_in_model"]
    .sum()
    .rename("n_snps_in_model_sum")
)

In [126]:
spredixcan_genes_sum_of_n_snps_in_model

gene_id
ENSG00000000419     2
ENSG00000000938    40
ENSG00000000971    44
ENSG00000001084    46
ENSG00000001167    48
                   ..
ENSG00000278540    44
ENSG00000278828     5
ENSG00000278845    91
ENSG00000281005    81
ENSG00000282608    40
Name: n_snps_in_model_sum, Length: 6444, dtype: int64

In [127]:
# add sum of snps in model to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_sum_of_n_snps_in_model
)

In [128]:
spredixcan_genes_models.shape

(6444, 8)

In [129]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_pc_variances_cov,tissues_variances,n_snps_used_sum,n_snps_in_model_sum
gene_id,,,,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,"[1.0442398458252409, 0.9557601541747592]","[0.013163142634155849, 0.00048574890347684763]","{'Brain_Hypothalamus': 0.013162153504206677, '...",2,2
ENSG00000000938,FGR,"(Breast_Mammary_Tissue, Esophagus_Mucosa, Pros...",36,"[31.63266933637128, 2.0783677518511436, 1.2716...",[0.17159486570780277],{'Breast_Mammary_Tissue': 0.002243819216617953...,40,40
ENSG00000000971,CFH,"(Breast_Mammary_Tissue, Stomach, Artery_Corona...",34,"[21.560769925683296, 7.3101709870454625, 1.837...","[0.2093060319587451, 0.058234347656008094, 0.0...","{'Breast_Mammary_Tissue': 0.01068561006340443,...",44,44
ENSG00000001084,GCLC,"(Pancreas, Artery_Coronary, Lung, Brain_Putame...",32,"[21.291939080546523, 4.699418748743197, 2.2489...","[0.39430527375123775, 0.11627488888534823, 0.0...","{'Pancreas': 0.015199520517598328, 'Artery_Cor...",46,46
ENSG00000001167,NFYA,"(Breast_Mammary_Tissue, Esophagus_Mucosa, Pros...",40,"[37.33566029549769, 1.3914019604538364]","[1.1642639979821048, 0.05742881287671711]","{'Breast_Mammary_Tissue': 0.03313519857456931,...",47,48


### Summarize prediction models for each gene

In [130]:
def _summarize_gene_models(gene_id):
    """
    For a given gene ID, it returns a dataframe with predictor SNPs in rows and tissues in columns, where
    values are the weights of SNPs in those tissues.
    It can contain NaNs.
    """
    gene_obj = spredixcan_gene_obj[gene_id]
    gene_tissues = spredixcan_genes_models.loc[gene_id, "tissue"]

    gene_models = {}
    gene_unique_snps = set()
    for t in gene_tissues:
        gene_model = gene_obj.get_prediction_weights(tissue=t, model_type=EQTL_MODEL)
        gene_models[t] = gene_model

        gene_unique_snps.update(set(gene_model.index))

    df = pd.DataFrame(
        data=np.nan, index=list(gene_unique_snps), columns=list(gene_tissues)
    )

    for t in df.columns:
        for snp in df.index:
            gene_model = gene_models[t]

            if snp in gene_model.index:
                df.loc[snp, t] = gene_model.loc[snp]

    return df

In [131]:
# testing
spredixcan_gene_obj["ENSG00000000419"].get_prediction_weights(
    tissue="Brain_Hypothalamus", model_type=EQTL_MODEL
)

varID
chr20_50862947_C_T_b38    0.431375
Name: weight, dtype: float64

In [132]:
spredixcan_gene_obj["ENSG00000000419"].get_prediction_weights(
    tissue="Brain_Substantia_nigra", model_type=EQTL_MODEL
)

varID
chr20_50957480_C_T_b38   -0.146796
Name: weight, dtype: float64

In [133]:
# # testing
# _gene_id = "ENSG00000000419"

# _gene_model = _summarize_gene_models(_gene_id)
# assert (
#     _gene_model.loc["chr20_50862947_C_T_b38", "Brain_Hypothalamus"].round(5) == 0.43138
# )
# assert pd.isnull(_gene_model.loc["chr20_50957480_C_T_b38", "Brain_Hypothalamus"])

# assert pd.isnull(_gene_model.loc["chr20_50862947_C_T_b38", "Brain_Substantia_nigra"])
# assert (
#     _gene_model.loc["chr20_50957480_C_T_b38", "Brain_Substantia_nigra"].round(5)
#     == -0.1468
# )

In [134]:
gene_models = {}

for gene_id in spredixcan_genes_models.index:
    gene_models[gene_id] = _summarize_gene_models(gene_id)

In [135]:
# # testing
# _gene_id = "ENSG00000000419"

# _gene_model = gene_models[_gene_id]
# assert (
#     _gene_model.loc["chr20_50862947_C_T_b38", "Brain_Hypothalamus"].round(5) == 0.43138
# )
# assert pd.isnull(_gene_model.loc["chr20_50957480_C_T_b38", "Brain_Hypothalamus"])

# assert pd.isnull(_gene_model.loc["chr20_50862947_C_T_b38", "Brain_Substantia_nigra"])
# assert (
#     _gene_model.loc["chr20_50957480_C_T_b38", "Brain_Substantia_nigra"].round(5)
#     == -0.1468
# )

In [136]:
# save
import gzip

with gzip.GzipFile(OUTPUT_DIR_BASE / "gene_tissues_models.pkl.gz", "w") as f:
    pickle.dump(gene_models, f)

In [137]:
# testing saved file
with gzip.GzipFile(OUTPUT_DIR_BASE / "gene_tissues_models.pkl.gz", "r") as f:
    _tmp = pickle.load(f)

In [138]:
assert len(gene_models) == len(_tmp)
assert gene_models["ENSG00000000419"].equals(_tmp["ENSG00000000419"])

### Count number of _unique_ SNPs predictors used and available across tissue models

In [139]:
def _count_unique_snps(gene_id):
    """
    For a gene_id, it counts unique SNPs in all models and their intersection with GWAS SNPs (therefore, used by S-PrediXcan).
    """
    gene_obj = spredixcan_gene_obj[gene_id]
    gene_tissues = spredixcan_genes_models.loc[gene_id, "tissue"]

    gene_unique_snps = set()
    for t in gene_tissues:
        t_snps = set(gene_models[gene_id].index)
        gene_unique_snps.update(t_snps)

    gene_unique_snps_in_gwas = gwas_variants_ids_set.intersection(gene_unique_snps)

    return pd.Series(
        {
            "unique_n_snps_in_model": len(gene_unique_snps),
            "unique_n_snps_used": len(gene_unique_snps_in_gwas),
        }
    )

In [140]:
# testing
spredixcan_genes_models[spredixcan_genes_models["n_snps_used_sum"] == 2].head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_pc_variances_cov,tissues_variances,n_snps_used_sum,n_snps_in_model_sum
gene_id,,,,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,"[1.0442398458252409, 0.9557601541747592]","[0.013163142634155849, 0.00048574890347684763]","{'Brain_Hypothalamus': 0.013162153504206677, '...",2,2
ENSG00000010256,UQCRC1,"(Thyroid, Whole_Blood)",2,"[1.1048350039574655, 0.8951649960425345]","[0.0008061424968110495, 0.0002308709788283639]","{'Thyroid': 0.0008025245652862433, 'Whole_Bloo...",2,2
ENSG00000017427,IGF1,"(Brain_Amygdala, Testis)",2,"[1.0389319526404415, 0.9610680473595585]","[0.02060052225760429, 0.003854854256224178]","{'Brain_Amygdala': 0.02059332044751194, 'Testi...",2,2
ENSG00000043093,DCUN1D1,"(Esophagus_Muscularis, Esophagus_Gastroesophag...",2,"[1.4951653523020276, 0.5048346476979725]","[0.0005196522562331344, 0.00014798630499321565]",{'Esophagus_Muscularis': 0.0004315621222418269...,2,2
ENSG00000081377,CDC14B,"(Brain_Nucleus_accumbens_basal_ganglia, Muscle...",2,"[1.1285007973441137, 0.8714992026558863]","[0.014647518608411533, 0.0074225002067654835]",{'Brain_Nucleus_accumbens_basal_ganglia': 0.01...,2,2


In [141]:
# case with two snps, not repeated across tissues
_gene_id = "ENSG00000000419"
display(
    spredixcan_gene_obj[_gene_id].get_prediction_weights(
        tissue="Brain_Hypothalamus", model_type=EQTL_MODEL
    )
)
display(
    spredixcan_gene_obj[_gene_id].get_prediction_weights(
        tissue="Brain_Substantia_nigra", model_type=EQTL_MODEL
    )
)

varID
chr20_50862947_C_T_b38    0.431375
Name: weight, dtype: float64

varID
chr20_50957480_C_T_b38   -0.146796
Name: weight, dtype: float64

In [142]:
# _tmp = _count_unique_snps(_gene_id)
# assert _tmp.shape[0] == 2
# assert _tmp["unique_n_snps_in_model"] == 2
# assert _tmp["unique_n_snps_used"] == 2

In [143]:
# get unique snps for all genes
spredixcan_genes_unique_n_snps = spredixcan_genes_models.groupby("gene_id").apply(
    lambda x: _count_unique_snps(x.name)
)

In [144]:
spredixcan_genes_unique_n_snps.head()

,unique_n_snps_in_model,unique_n_snps_used
gene_id,,
ENSG00000000419,2,2
ENSG00000000938,5,5
ENSG00000000971,12,12
ENSG00000001084,23,23
ENSG00000001167,14,13


In [145]:
assert (
    spredixcan_genes_unique_n_snps["unique_n_snps_in_model"]
    >= spredixcan_genes_unique_n_snps["unique_n_snps_used"]
).all()

In [146]:
# add unique snps to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(spredixcan_genes_unique_n_snps)

In [147]:
spredixcan_genes_models.shape

(6444, 10)

In [148]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_pc_variances_cov,tissues_variances,n_snps_used_sum,n_snps_in_model_sum,unique_n_snps_in_model,unique_n_snps_used
gene_id,,,,,,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,"[1.0442398458252409, 0.9557601541747592]","[0.013163142634155849, 0.00048574890347684763]","{'Brain_Hypothalamus': 0.013162153504206677, '...",2,2,2,2
ENSG00000000938,FGR,"(Breast_Mammary_Tissue, Esophagus_Mucosa, Pros...",36,"[31.63266933637128, 2.0783677518511436, 1.2716...",[0.17159486570780277],{'Breast_Mammary_Tissue': 0.002243819216617953...,40,40,5,5
ENSG00000000971,CFH,"(Breast_Mammary_Tissue, Stomach, Artery_Corona...",34,"[21.560769925683296, 7.3101709870454625, 1.837...","[0.2093060319587451, 0.058234347656008094, 0.0...","{'Breast_Mammary_Tissue': 0.01068561006340443,...",44,44,12,12
ENSG00000001084,GCLC,"(Pancreas, Artery_Coronary, Lung, Brain_Putame...",32,"[21.291939080546523, 4.699418748743197, 2.2489...","[0.39430527375123775, 0.11627488888534823, 0.0...","{'Pancreas': 0.015199520517598328, 'Artery_Cor...",46,46,23,23
ENSG00000001167,NFYA,"(Breast_Mammary_Tissue, Esophagus_Mucosa, Pros...",40,"[37.33566029549769, 1.3914019604538364]","[1.1642639979821048, 0.05742881287671711]","{'Breast_Mammary_Tissue': 0.03313519857456931,...",47,48,14,13


### Save

In [149]:
# this is important, other scripts depend on gene_name to be unique
assert spredixcan_genes_models["gene_name"].is_unique

In [150]:
assert not spredixcan_genes_models.isna().any(None)

In [151]:
spredixcan_genes_models.to_pickle(OUTPUT_DIR_BASE / "gene_tissues.pkl")